In [ ]:
def plotting_for_sweep(plotmeas, last_plot_update, plot_update_time, plot_axes, first_time):
    if first_time:
        plot_update_start = time.time()
        first_update = time.time()
        plot_axes = plotting.plot_measurement(plotmeas)
        plot_update_time = plot_update_start - first_update 
    else:
        plot_update_start = time.time()
        if plot_update_start - last_plot_update > 10*plot_update_time:	
            plotting.update_plot_measurement(plotmeas, plot_axes)
            print('I was called')
        last_plot_update = time.time()
    #print(plot_axes)
    return last_plot_update, plot_update_time, plot_axes

import itertools

def sweep1(measurer, *params, filename=None, root_dir=None, plot=True, header=None, output=True, save = True, time_war_label=True,cycle_save_mode=False):
	import warnings
	warnings.filterwarnings("ignore",".*Using default event loop until function specific to this GUI is implemented*")
	if (time_war_label):
		try:
			from ipywidgets import HTML
			from IPython.display import display
			sweep_state_widget = HTML()
			display(sweep_state_widget)
		except:
			sweep_state_widget = None
	
		print("Started at: ", time.strftime("%b %d %Y %H:%M:%S", time.localtime()))
		start_time = time.time()
	
	# extracting sweep parameters from list
	# check if it is a list of pairs
	# param looks like (voltages, lambda x: set_voltage(x, channel_id), 'Coil voltage{}'.format(coil)), so
	# param[0] = voltages, param[2] = 'Coil voltage'
	# Here everything is OK
	sweep_dim = tuple([len(param[0]) for param in params])
	sweep_dim_vals = tuple([param[0] for param in params])
	sweep_dim_names = tuple([param[2] if len(param)>2 else 'param_{0}'.format(param_id) for param_id, param in enumerate(params)])
	sweep_dim_units = tuple([param[3] if len(param)>3 else '' for param_id, param in enumerate(params)])
	sweep_dim_setters = tuple(param[1] for param in params)
	
	# determining return type of measurer: in could be 2 traces (im, re)
	# Here everything is OK
	point_types = measurer.get_dtype()           # ex: {'S-parameter': complex}
	meas_opts = measurer.get_opts()              # ex: {'S-parameter': {'log': 20}}
	points = measurer.get_points()               # ex: {'S-parameter': [('Frequency', array([6.194645e+09]), 'Hz')]}             should return dict of meas_name:points
	# points items: dict_items([('S-parameter', [('Frequency', array([6.194645e+09]), 'Hz')])])   
	# mname = 'S-parameter'
	point_dim_names = {mname:tuple([cname	   for cname, cvals, cunits in mxvals]) for mname, mxvals in points.items()} # 'Frequency'
	point_dim		= {mname:tuple([len(cvals) for cname, cvals, cunits in mxvals]) for mname, mxvals in points.items()} # 1
	point_dim_vals	= {mname:tuple([cvals	   for cname, cvals, cunits in mxvals]) for mname, mxvals in points.items()} # 6.195
	point_dim_units	= {mname:tuple([cunits	   for cname, cvals, cunits in mxvals]) for mname, mxvals in points.items()} # 'Hz'
	# initializing empty ndarrays for measurment results
	data = {mname: np.zeros(sweep_dim + point_dim[mname], dtype=point_types[mname])*np.nan for mname, mxvals in points.items()}
	
	def non_unity_dims(mname):
		return tuple(dim for dim in data[mname].shape if dim > 1)
		
	def non_unity_dim_names(mname):
		all_dim_names = sweep_dim_names + point_dim_names[mname]
		return tuple(dim_name for dim, dim_name in zip(data[mname].shape, all_dim_names) if dim > 1)

	def non_unity_dim_vals(mname):
		all_dim_vals = sweep_dim_vals + point_dim_vals[mname]
		return tuple(dim_value for dim, dim_value in zip(data[mname].shape, all_dim_vals) if dim > 1)
		
	def non_unity_dim_units(mname):
		all_dim_units = sweep_dim_units + point_dim_units[mname]
		return tuple(dim_unit for dim, dim_unit in zip(data[mname].shape, all_dim_units) if dim > 1)
	
	mk_measurement = lambda x: {mname: (non_unity_dim_names(mname), non_unity_dim_vals(mname), 
					  np.reshape(data[mname], non_unity_dims(mname)),
					  meas_opts[mname], non_unity_dim_units(mname)) for mname in points.keys()}
	
	import os
	last_plot_update, plot_update_time, plot_axes = plotting_for_sweep(mk_measurement(0), time.time(), 0, {}, True)
	start_time = time.time()
	
	if (time_war_label):
	# starting sweep	
		print(0)#sweep_state_print("First sweep...",sweep_state_widget)
	try:
		# looping over all indeces at the same time
		all_indeces = itertools.product(*([i for i in range(d)] for d in sweep_dim))
		vals = [None for d in sweep_dim] # set initial parameter vals to None
		done_sweeps = 0
		total_sweeps = np.prod([d for d in sweep_dim])
		for indeces in all_indeces:
			# check which values have changed this sweep
			old_vals = vals
			vals = [params[param_id][0][val_id] for param_id, val_id in enumerate(indeces)]
			changed_vals = [old_val!=val for old_val, val in zip(old_vals, vals)]
			# set to new param vals
			for val, setter, changed in zip(vals, sweep_dim_setters, changed_vals):
				if changed:
					print('I change here the settings')#setter(val)
			#measuring
			mpoint = measurer.measure()
			for mname in points.keys():
				data[mname][[(i) for i in indeces]+[...]] = mpoint[mname]
				mpoint_m_flat = np.reshape(mpoint[mname], tuple(dim for dim in point_dim[mname] if dim > 1)) #this is for ASCII
			
			last_plot_update, plot_update_time, plot_axes = plotting_for_sweep(mk_measurement(0), last_plot_update, plot_update_time, plot_axes, False)
			
			done_sweeps += 1
			avg_time = (time.time() - start_time)/done_sweeps # average time for one sweep
			# What is going on here? And for which purpose?
			param_val_mes = ',\t'.join(['{0}: {1:6.4g}'.format(param[2], val) for param,val in zip(params,vals)])
			stat_mes_fmt = '\rTime left: {0},\tparameter values: {1},\taverage cycle time: {2:4.2g}s\t'
			stat_mes = stat_mes_fmt.format(format_time_delta(avg_time*(total_sweeps-done_sweeps)), param_val_mes, avg_time)
			if (time_war_label):
				sweep_state_print(stat_mes,sweep_state_widget)
		if (time_war_label):
			print("\nElapsed time: " + format_time_delta(time.time() - start_time))
	
	finally:	
		for mname in points.keys():
			header = {'name': filename, 'type': mname, 'params':non_unity_dim_names(mname)}
	save_hdf(mk_measurement(0), filename, points, sweep_dim_names, point_dim_names, sweep_dim_vals, point_dim_vals)
	#print(mk_measurement(0))
	#print(mk_measurement(0)['S-parameter'][2])
	return mk_measurement(0)
	

def save_hdf(measurement, filename, points, sweep_dim_names, points_dim_names, sweep_dim_vals, point_dim_vals):
	import h5py
	parameters = []
	all_arrays = []
	with h5py.File(filename + '.hdf5', 'w') as f:
		for mname in points.keys():
			#print(str(mname))
			parameters.append(f.create_group(str(mname)))
			for sweep_name, sweep_value in zip(measurement[mname][0], measurement[mname][1]):
				#print(sweep_name, sweep_value)
				#print(sweep_dim_vals[index])
				all_arrays.append(parameters[len(parameters)-1].create_dataset(str(sweep_name), data=sweep_value, compression='gzip'))
			#print(mname)
			print(measurement[mname][2])
			all_arrays.append(parameters[len(parameters)-1].create_dataset('Parameter values', data=measurement[mname][2], compression='gzip'))
		metadata = {'Date': time.time(),'Power': '','VNA power': ''}
		f[mname].attrs.update(metadata)
		print(f[mname].attrs.items())

In [ ]:
pna.set_nop(10)
voltages = np.arange(0, 1, 0.5)
result = sweep1(pna, (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil voltage'), 
                    filename='ExampleHDF5')

In [ ]:
def read_hdf(filename):
    result = {}
    with h5py.File(filename, 'r') as f:
        for param in f.keys():
            print(f[param].attrs.items())
            variables = []
            variables_values = []
            for variable in f[str(param)].keys():
                #print(variable)
                if variable != 'Parameter values':
                    variables.append(variable)
                variables_values.append(f[str(param) + '/' + str(variable)][:])
            #variables_values.append(f[str(param) + '/' + str(param)][:])
            result[param] = (tuple(variables), tuple(variables_values))
    #print(result)
    return result

In [ ]:
read_hdf('ExampleHDF5.hdf5')